In [1]:
import re
import os
import pandas as pd

In [2]:
def custo_por_mao(big_blind,small_blind,ante,numero_de_jogadores):
    return (big_blind + small_blind + ante*numero_de_jogadores)/numero_de_jogadores

def convert_k_to_numeric(df, column_name):
    """
    Converts values in a specified column from 'K' notation to numeric.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the column to be converted.
    column_name (str): The name of the column to convert.

    Returns:
    pd.DataFrame: The DataFrame with the specified column converted.
    """
    def convert(value):
        if isinstance(value, str) and 'K' in value:
            return int(float(value.replace('K', '')) * 1000)
        return value

    df[column_name] = pd.to_numeric(df[column_name].apply(convert))
    return df

In [3]:
blind_structure = pd.read_csv('./1_dolar_45_players_sit_go.csv')
blind_structure[['small_blind','big_blind']] = blind_structure['Blinds'].str.split('/',expand=True)
blind_structure.set_index('Level',inplace=True)

blind_structure = convert_k_to_numeric(blind_structure,'Ante')
blind_structure = convert_k_to_numeric(blind_structure,'small_blind')
blind_structure = convert_k_to_numeric(blind_structure,'big_blind')
blind_structure['Minutes'] = pd.to_numeric(blind_structure['Minutes'])

#### A cada volta completa na mesa de tamanho 9 numero de jogadores no torneio o custo médio por mão é dado por:

#### (N * Ante + bigblind + smalllind)/N ou Ante + (bigblind + smalllind)/N

In [4]:
blind_structure['custo_medio_por_mao'] = blind_structure['Ante'] + (blind_structure['small_blind'] + blind_structure['big_blind'])/9
blind_structure['Tempo Acumulado'] = blind_structure['Minutes'].cumsum()

In [5]:
df_transformed = pd.DataFrame({
    'Level': blind_structure.index.repeat(10).values,
    'Value': range(1, blind_structure['Tempo Acumulado'].max() + 1)
}).set_index("Level")

In [6]:
blind_structure_extendido = pd.merge(df_transformed,blind_structure,left_index=True,right_index=True,how='left')

In [7]:
blind_structure_extendido = blind_structure_extendido.drop("Tempo Acumulado",axis=1)

In [291]:
blind_structure_extendido['maos_jogadas_por_minuto'] = 60/50

In [292]:
blind_structure_extendido['custo_por_minuto'] = blind_structure_extendido['maos_jogadas_por_minuto'] * blind_structure_extendido['custo_medio_por_mao']

In [293]:
blind_structure_extendido['custo_acumulado'] = blind_structure_extendido.custo_por_minuto.cumsum()

In [447]:
custo_final = blind_structure_extendido[blind_structure_extendido['Value'] == 130].iloc[0]['custo_acumulado']
custo_inicial = blind_structure_extendido[blind_structure_extendido['Value'] >= 104].iloc[0]['custo_acumulado']

In [448]:
quantidade_necessaria_de_fichas = custo_final - custo_inicial

In [451]:
quantidade_necessaria_de_fichas - 6100

1240.0